### Installing the required Modules


In [2]:
# Importing the require modules
import numpy as np

import os
import pickle 
import blosc

import re
from pprint import pprint

from openbabel import pybel
from openbabel import openbabel as ob

error_files = []

## Helper function filter to extract all the bond lengths, angles and dihedrals from a molecule

In [3]:
def filterAtom(atom: str) -> str:
    '''Removes any number from the string e.g. C3 becomes C'''
    pattern_order = r'[0-9]'
    return re.sub(pattern_order, '', atom)

def atomType(mol, atomIdx) -> str:
    '''get the atomic type given an atom index'''
    return mol.OBMol.GetAtom(atomIdx).GetType()

In [4]:
def getBonds(mol) -> dict:
    '''Iterate through all the bonds in a molecule'''
    
    bonds = {}
    
    for bond in ob.OBMolBondIter(mol.OBMol):
        begin = filterAtom(atomType(mol, bond.GetBeginAtomIdx()))
        end = filterAtom(atomType(mol, bond.GetEndAtomIdx()))
        bond_length = round(bond.GetLength(),4)
        bond_order = bond.GetBondOrder()
        
        # Swap them for lexographic order
        if (end < begin):
            begin, end = end, begin
        
        #Appending to the dictionary of list
        if (f"{begin} - {end} , {bond_order}" in bonds.keys()):
            bonds[f"{begin} - {end} , {bond_order}"].append(bond_length)
        else:
            bonds[f"{begin} - {end} , {bond_order}"]=[bond_length]
        
    return bonds

In [5]:
def getAngles(mol) -> dict:
    '''Iterate through all the bond angles in a molecule'''
    
    angles = {}
    
    for angle in ob.OBMolAngleIter(mol.OBMol):
        a = angle[0] + 1
        b = mol.OBMol.GetAtom(angle[1] + 1)
        c = angle[2] + 1
        bond_angle = b.GetAngle(a, c)

        aType = filterAtom(atomType(mol, a))
        bType = filterAtom(b.GetType())
        cType = filterAtom(atomType(mol, c))
        
        # Swap them for lexographic order
        if (cType < aType):
            aType, cType = cType, aType
            
        #Appedning to the dictonary of lists if the angle b/t the elements already existse
        if(f"{aType} - {bType} - {cType}" in angles.keys()):
            angles[f"{aType} - {bType} - {cType}"].append(round(bond_angle ,3))
        else:
            angles[f"{aType} - {bType} - {cType}"] = [round(bond_angle ,3)]                                                                                      
                                                                                                     
    return angles

In [6]:
def getTorsions(mol) -> dict:
    '''Iterate through all the torsions in a molecule'''
    
    torsions = {}
    
    for torsion in ob.OBMolTorsionIter(mol.OBMol):
        a = torsion[0] + 1
        b = torsion[1] + 1
        c = torsion[2] + 1
        d = torsion[3] + 1
        torsion_angle = mol.OBMol.GetTorsion(a, b, c, d)

        aType = filterAtom(atomType(mol, a))
        bType = filterAtom(atomType(mol, b))
        cType = filterAtom(atomType(mol, c))
        dType = filterAtom(atomType(mol, d))

        # Switch if not in lexographic order
        if(dType < aType):
            aType, dType = dType, aType
            
        #Appedning to the dictonary of lists if the torsion angle b/t the elements already existse
        if(f"{aType} - {bType} - {cType} -{dType}" in torsions.keys()):
            torsions[f"{aType} - {bType} - {cType} -{dType}"].append(round(torsion_angle, 3))
        
        # Checking for palindromic sequence
        elif(f"{dType} - {cType} - {bType} -{aType}" in torsions.keys()):
            torsions[f"{dType} - {cType} - {bType} -{aType}"].append(round(torsion_angle, 3))
            
        else:
            torsions[f"{aType} - {bType} - {cType} -{dType}"] = [round(torsion_angle, 3)]
    
    return torsions

### Wrapping all the helper functions into a main function

In [7]:
def analyze_molecular_data(file: str) -> (dict, dict, dict):
    '''Fetches the bond lengths, angles and dihedrals from a given file'''
    
    # Load the File
    extension=file.split('.')[-1]
    try:
        mol = next(pybel.readfile(extension, file))
    except:
        error_files.append(file)
        return ({},{},{})

    # Get bond lengths
    bond_lengths = getBonds(mol)
    
    # Get bond angles
    bond_angles = getAngles(mol)
    
    # Get torsions
    dihedrals = getTorsions(mol)

    return (bond_lengths, bond_angles, dihedrals)


def mergeDictionaries(dict1 : dict, dict2: dict):
    '''Merges two Python dictionary by combining elements for a common key
    The function merges dict2 into dict 1'''
    for key, value in dict2.items():
        if key in dict1:
            dict1[key].extend(value)
        else:
            dict1[key] = value

### Testing the above functions on a sample molecule

In [8]:
# Example usage
xyz_file = '21395061/cod-crest/W/WKGCLMYDEAXJHN-UHFFFAOYSA-N.xyz'
#mol = next(pybel.readfile('xyz', xyz_file))
#pprint(getBonds(mol))
#pprint(getAngles(mol))
#pprint(getTorsions(mol))

molecular_data = analyze_molecular_data(xyz_file)
pprint(molecular_data)

({'C - C , 1': [1.5238, 1.5268, 1.5256, 1.525, 1.525, 1.5256, 1.5268, 1.5238],
  'C - H , 1': [1.0878,
                1.0927,
                1.0873,
                1.0879,
                1.0921,
                1.0995,
                1.0927,
                1.0921,
                1.0914,
                1.0996,
                1.0914,
                1.0914,
                1.0996,
                1.0921,
                1.0914,
                1.0995,
                1.0927,
                1.0921,
                1.0927,
                1.0879,
                1.0873,
                1.0878],
  'C - O , 1': [1.4109, 1.4109],
  'Car - Car , 1': [1.402, 1.4263, 1.4095, 1.4095, 1.4263, 1.402],
  'Car - Car , 2': [1.374, 1.3649, 1.4143, 1.3649, 1.374],
  'Car - H , 1': [1.0787, 1.0802, 1.0778, 1.0778, 1.0802, 1.0787],
  'Car - O , 1': [1.3616, 1.3616]},
 {'C - C - C': [111.557, 112.053, 111.71, 111.554, 112.048, 111.706],
  'C - C - H': [109.969,
                109.994,
          

## Extracting Molecular Data from all the files

In [9]:
#Collecting data from the Molecular Dataset

bond_lengths = {}
bond_angles = {}
dihedrals = {}

ext = ('.xyz',)
count = 0

for root, dirs, files in os.walk('21395061/cod-crest/'):
    for file in files:
        extension=os.path.splitext(file)[1]
        count +=1
        if(extension not in ext): continue
            
        filename = os.path.join(root,file)
        length, angle, torsion = analyze_molecular_data(filename)
        
        mergeDictionaries(bond_lengths, length)
        mergeDictionaries(bond_angles, angle)
        mergeDictionaries(dihedrals, torsion)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -59.433677868596          0.000316298731)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -79.377053843214          0.000924825445)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -86.034286802623          0.000854184241)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -58.758444780557          0.000328546459)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -111.085364902439          0.000603406375)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -65.485149618314          0.000041819052)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -84.893951507837          0.000432179138)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -43.338442248229          0.000158257975)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -161.206963397959          0.000759112675)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -29.282164042213          0.000830019973)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -33.667170665269          0.000397739222)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -86.420450608817          0.000324359427)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -74.997605752354          0.000845558168)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -165.771494372887          0.000791349152)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -48.571198361008          0.000925247483)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -82.772960815588          0.000458128560)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -60.644727690321          0.000020041702)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -57.446200193895          0.000553047190)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -44.312761534497          0.000064360298)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -177.731198219616          0.000070578272)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -33.308841946164          0.000039686577)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -73.268435620933          0.000040479521)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -40.117650282955          0.000059132952)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -83.628260032053          0.000061706538)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -98.444347325926          0.000046259000)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

...

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -47.435868878285          0.000665039542)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -66.798978199926          0.000029508873)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -62.144238995291          0.000916279813)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -88.804784719799          0.000614162326)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -84.793896070338          0.000616788948)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -82.029977237774          0.000706224090)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -33.945870419655          0.000061831930)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -74.422943126984          0.000051085024)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -54.680564784769          0.000154549806)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -112.699031862537          0.000459469507)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -69.503084069163          0.000377972680)

*** Open Babel Warning  in ReadMolecule
  Problems reading an XYZ file: Cannot read the first line.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -117.790530459887          0.000519796814)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -101.750431148934          0.000628356293)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -52.940275663431          0.000654169263)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -97.644207344361          0.000460317869)



*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -69.239229636864          0.000678304334)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -166.784254555298          4.120316638877)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -37.790921688986          0.000848148177)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -82.253339242296          0.000496082464)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -55.997689097979          0.000073604696)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::Perceive

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -54.232683292412          0.000543879746)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -65.208630930994          0.000041803273)

*** Open Babel Warning  in ReadMolecule
  Problems reading an XYZ file: Cannot read the first line.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -52.466823350961          0.000762808739)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done       -110.037414563866          0.000456640074)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is SCF done        -59.770848497959          0.000390069976)



### Running the corrupt xyz files with corresponding sdf files and writing the bond length, angle, torsion data as pickle object

In [10]:
#Trying with sfd files for the corrupted xyz files

for file in error_files:
    filename = os.path.splitext(file)[0] + '.sdf'
    length, angle, torsion = analyze_molecular_data(filename)
    mergeDictionaries(bond_lengths, length)
    mergeDictionaries(bond_angles, angle)
    mergeDictionaries(dihedrals, torsion)

In [14]:
def MolecularDataWriter_compress():
    
    pickled_bond_lengths = pickle.dump(bond_lengths)
    compressed_bond_lengths = blosc.compress(pickled_bond_lengths)
    
    with open('cod-crest_bond_lengths.dat', 'wb') as f:
        f.write(compressed_bond_lengths)
        
    pickled_bond_angles = pickle.dump(bond_angles)
    compressed_bond_angles = blosc.compress(pickled_bond_angles)
        
    with open('cod-crest_bond_angles.dat', 'wb') as f:
        f.write(compressed_bond_angles)
        
    pickled_torsions = pickle.dump(torsion)
    compressed_dihedrals = blosc.compress(pickled_torsions)
        
    with open('cod-crest_torsions.dat', 'wb') as f:
        f.write(compressed_dihedrals)

In [17]:
def MolecularDataWriter_pickle():
    
    with open('cod-crest_bond_lengths.pkl', 'wb') as f:
        pickle.dump(bond_lengths, f)
        
    with open('cod-crest_bond_angles.pkl', 'wb') as f:
        pickle.dump(bond_angles, f)
        
    with open('cod-crest_torsions.pkl', 'wb') as f:
        pickle.dump(dihedrals, f)

In [18]:
MolecularDataWriter_pickle()